In [ ]:
from tools.tools import get_path, get_year
import shutil
import os
import xarray as xr
import numpy_financial as npf

import tools.config as config

def amortize_costs(origin_path_name, target_path_name, amortize_file, years, rate=0.07, horizon=30):
    """
    计算成本均摊并保存新的成本数组。

    参数：
    path_name: str - 文件路径
    file_name: str - 文件名
    rate: float - 利率，默认为0.07
    horizon: int - 时间跨度，默认为30年
    """
    # 初始化一个空字典来存储所有年份的成本数据
    print("Start to calculate cost amortize.")
    
    costs = {}
    for year in years:
        full_file_name = os.path.join(origin_path_name, f'out_{year}', f'{amortize_file}_{year}.nc')
        # 从 Dataset 中提取出包含成本数据的那个 DataArray
        costs[year] = xr.open_dataset(full_file_name)['data']

    # 我们需要一个模板来确定新数组的形状和坐标，用第一个年份的即可
    template_da = costs[years[0]]
    new_costs = {year: xr.zeros_like(template_da) for year in years}
    
    for year in years:
        # pv_values 现在是一个 DataArray，可以直接用于计算
        pv_values = costs[year]
        
        # npf.pmt 会自动在 DataArray 的每个元素上计算，并返回一个新的 DataArray
        annual_payment = -1 * npf.pmt(rate, horizon, pv=pv_values, fv=0, when='begin')
        
        # 将计算出的年金 (一个 DataArray) 分配到未来年份
        for offset in range(horizon):
            target_year = year + offset
            if target_year in years:  # 确保目标年份在我们的处理范围内
                # 两个结构相同的 DataArray 可以直接相加！
                new_costs[target_year] += annual_payment
    
    for year in years:
        target_year_path = os.path.join(target_path_name, str(year))
        dst_file = os.path.join(target_year_path, f'{amortize_file}_amortised_{year}.nc')
        new_costs[year].to_netcdf(dst_file)

input_files = config.INPUT_FILES
path_name_0 = get_path(input_files[2])
path_name_1 = get_path(input_files[1])
path_name_2 = get_path(input_files[0])

output_path = f'{config.TASK_DIR}/carbon_price/0_base_data'
os.makedirs(output_path, exist_ok=True)

In [25]:
copy_files = ['xr_biodiversity_GBF2_priority_ag_management','xr_biodiversity_GBF2_priority_non_ag','xr_cost_ag','xr_cost_agricultural_management','xr_cost_non_ag','xr_cost_transition_ag2ag','xr_GHG_ag_management','xr_GHG_non_ag','xr_revenue_ag','xr_revenue_agricultural_management','xr_revenue_non_ag','xr_transition_cost_ag2non_ag','xr_transition_GHG']
amortize_files = ['xr_transition_cost_ag2non_ag']
diff_files = ['xr_biodiversity_GBF2_priority_ag','xr_GHG_ag.nc']

In [ ]:
for i in range(0,3):
    origin_path_name = get_path(input_files[i])
    target_path_name = os.path.join(output_path, input_files[i])
    years = get_year(origin_path_name)
    amortize_costs(origin_path_name, target_path_name, amortize_files[0], years)
    for year in years:
        year_path = os.path.join(origin_path_name, f'out_{year}')
        target_year_path = os.path.join(target_path_name, str(year))
        os.makedirs(target_year_path, exist_ok=True)
        for copy_file in copy_files:
            if year == 2010:
                if copy_file == 'xr_transition_GHG':
                    continue
                
            src_file = os.path.join(year_path, f'{copy_file}_{year}.nc')
            dst_file = os.path.join(target_year_path, f'{copy_file}_{year}.nc')
            shutil.copy(src_file, dst_file)
        if year > years[0]:
            for diff_file in diff_files:
                if diff_file== 'xr_biodiversity_GBF2_priority_ag':
                    year_path_0 = os.path.join(origin_path_name, f'out_{year-1}')
                    src_file_0 = os.path.join(year_path_0, diff_file + f'_{year-1}.nc')
                    src_file_1 = os.path.join(year_path, diff_file + f'_{year}.nc')
                    ds_0 = xr.open_dataset(src_file_0)
                    ds_1 = xr.open_dataset(src_file_1)
                    ds_res = ds_1 - ds_0
                    dst_file = os.path.join(target_year_path, f'{diff_file}_{year}.nc')
                elif diff_file == 'xr_GHG_ag.nc':
                    year_path_0 = os.path.join(origin_path_name, f'out_{year-1}')
                    src_file_0 = os.path.join(year_path_0, diff_file)
                    src_file_1 = os.path.join(year_path, diff_file)
                    ds_0 = xr.open_dataset(src_file_0)
                    ds_1 = xr.open_dataset(src_file_1)
                    ds_res = ds_0 - ds_1
                    dst_file = os.path.join(target_year_path, diff_file)
                else:
                    print(f'Error: {diff_file} not found')
                ds_res.to_netcdf(dst_file)
    

Start to calculate cost amortize.


In [4]:
origin_path_name = get_path(input_files[0])
target_path_name = os.path.join(output_path, input_files[0])
years = get_year(origin_path_name)
amortize_file = amortize_files[0]

In [5]:
costs = {}

for year in years:
    full_file_name = os.path.join(origin_path_name, f'out_{year}', f'{amortize_file}_{year}.nc')
    costs[year] = xr.open_dataset(full_file_name)

In [27]:
a = xr.open_dataset(full_file_name)

In [14]:
amortize_file

'xr_transition_cost_ag2non_ag'

In [8]:
xr.open_dataset(full_file_name)

<xarray.Dataset> Size: 8GB
Dimensions:            (From water-supply: 2, cell: 468686, From land-use: 28,
                        To land-use: 9, Cost type: 8)
Coordinates:
  * From water-supply  (From water-supply) <U3 24B 'dry' 'irr'
  * cell               (cell) int64 4MB 0 1 2 3 ... 468682 468683 468684 468685
  * From land-use      (From land-use) <U27 3kB 'Apples' ... 'Winter oilseeds'
  * To land-use        (To land-use) <U29 1kB 'BECCS' ... 'Sheep Carbon Plant...
  * Cost type          (Cost type) <U34 1kB 'Establishment cost (Ag2Non-Ag)' ...
Data variables:
    data               (From water-supply, cell, From land-use, To land-use, Cost type) float32 8GB ...

In [9]:
print(a)

<xarray.Dataset> Size: 8GB
Dimensions:            (From water-supply: 2, cell: 468686, From land-use: 28,
                        To land-use: 9, Cost type: 8)
Coordinates:
  * From water-supply  (From water-supply) <U3 24B 'dry' 'irr'
  * cell               (cell) int64 4MB 0 1 2 3 ... 468682 468683 468684 468685
  * From land-use      (From land-use) <U27 3kB 'Apples' ... 'Winter oilseeds'
  * To land-use        (To land-use) <U29 1kB 'BECCS' ... 'Sheep Carbon Plant...
  * Cost type          (Cost type) <U34 1kB 'Establishment cost (Ag2Non-Ag)' ...
Data variables:
    data               (From water-supply, cell, From land-use, To land-use, Cost type) float32 8GB ...


In [13]:
list(a.data_vars)[0]

'data'

In [18]:
rate=0.07
horizon=30

costs = {}
for year in years:
    full_file_name = os.path.join(origin_path_name, f'out_{year}', f'{amortize_file}_{year}.nc')
    # 从 Dataset 中提取出包含成本数据的那个 DataArray
    costs[year] = xr.open_dataset(full_file_name)['data']

# 我们需要一个模板来确定新数组的形状和坐标，用第一个年份的即可
template_da = costs[years[0]]
new_costs = {year: xr.zeros_like(template_da) for year in years}

for year in years:
    # pv_values 现在是一个 DataArray，可以直接用于计算
    pv_values = costs[year]
    
    # npf.pmt 会自动在 DataArray 的每个元素上计算，并返回一个新的 DataArray
    annual_payment = -1 * npf.pmt(rate, horizon, pv=pv_values, fv=0, when='begin')
    
    # 将计算出的年金 (一个 DataArray) 分配到未来年份
    for offset in range(horizon):
        target_year = year + offset
        if target_year in years:  # 确保目标年份在我们的处理范围内
            # 两个结构相同的 DataArray 可以直接相加！
            new_costs[target_year] += annual_payment

for year in years:
    target_year_path = os.path.join(target_path_name, str(year))
    dst_file = os.path.join(target_year_path, f'{amortize_file}_amortised_{year}.nc')
    new_costs[year].to_netcdf(dst_file)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'rasterio']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [19]:
full_file_name

'../../../output/20250822_Paper2_Results/Run_1_GHG_high_BIO_high/output\\2025_08_22__13_22_53_RF3_2010-2050\\out_2019\\xr_transition_cost_ag2non_ag_2019.nc'

In [20]:
xr.open_dataset(full_file_name)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'rasterio']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html